In [2]:
import pandas as pd
import csv
from pathlib import Path
import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import hvplot.pandas as hvplot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [3]:
tickers_data = Path("senators_fixed_dates_&amounts.csv")
tickers_data_df = pd.read_csv(tickers_data, infer_datetime_format=True, parse_dates=True)

tickers_data_df

,transaction_date,owner,ticker,asset_type,type,amount,state,industry,senator
0,4/18/2023,Spouse,ESS,Stock,Sale (Full),"$15,000",RI,Real Estate Investment Trusts,Sheldon Whitehouse
1,4/18/2023,Self,ESS,Stock,Sale (Full),"$15,000",RI,Real Estate Investment Trusts,Sheldon Whitehouse
2,5/16/2023,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Michael F. Bennet
3,4/4/2023,Spouse,UPS,Stock,Sale (Full),"$15,000",WV,Trucking Freight/Courier Services,Shelley Moore Capito
4,4/4/2023,Spouse,MCD,Stock,Sale (Partial),"$15,000",WV,Restaurants,Shelley Moore Capito
...,...,...,...,...,...,...,...,...,...
8441,8/17/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Sheldon Whitehouse
8442,8/16/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Pat Roberts
8443,8/15/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Rob Portman
8444,8/2/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Thomas R. Carper


Find the specific tickers that each senator is trading and bring in data for those tickers

Senator, date, amount, ticker, type

In [4]:
tickers_df_dropped=tickers_data_df.dropna()
tickers_df_dropped.head

<bound method NDFrame.head of      transaction_date   owner ticker asset_type            type     amount  \
0           4/18/2023  Spouse    ESS      Stock     Sale (Full)   $15,000    
1           4/18/2023    Self    ESS      Stock     Sale (Full)   $15,000    
3            4/4/2023  Spouse    UPS      Stock     Sale (Full)   $15,000    
4            4/4/2023  Spouse    MCD      Stock  Sale (Partial)   $15,000    
5            4/4/2023  Spouse    DUK      Stock     Sale (Full)   $50,000    
...               ...     ...    ...        ...             ...        ...   
7627        1/16/2015    Self    NOG      Stock        Purchase   $50,000    
7628         1/6/2015    Self    FCX      Stock     Sale (Full)  $100,000    
7640       12/12/2014  Spouse    XOM      Stock     Sale (Full)   $15,000    
7641       12/19/2014  Spouse     BA      Stock        Purchase   $15,000    
7642       12/12/2014  Spouse    BAC      Stock     Sale (Full)   $15,000    

     state                       

In [5]:
tickers_df=pd.DataFrame(tickers_df_dropped)
# tickers_df

tickers_df=tickers_df[['transaction_date', 'ticker', 'type', 'amount', 'senator']]
display(tickers_df)

,transaction_date,ticker,type,amount,senator
0,4/18/2023,ESS,Sale (Full),"$15,000",Sheldon Whitehouse
1,4/18/2023,ESS,Sale (Full),"$15,000",Sheldon Whitehouse
3,4/4/2023,UPS,Sale (Full),"$15,000",Shelley Moore Capito
4,4/4/2023,MCD,Sale (Partial),"$15,000",Shelley Moore Capito
5,4/4/2023,DUK,Sale (Full),"$50,000",Shelley Moore Capito
...,...,...,...,...,...
7627,1/16/2015,NOG,Purchase,"$50,000",John Hoeven
7628,1/6/2015,FCX,Sale (Full),"$100,000",John Hoeven
7640,12/12/2014,XOM,Sale (Full),"$15,000",Pat Roberts
7641,12/19/2014,BA,Purchase,"$15,000",Pat Roberts


In [6]:
tickers_counts=tickers_df['ticker'].value_counts()
msft_counts=tickers_counts['MSFT']


msft_counts

96

In [7]:
most_common_ticker=tickers_df['ticker'].value_counts().head(5)
most_common_ticker

ticker
AAPL    168
MSFT     96
BAC      81
DIS      79
PFE      77
Name: count, dtype: int64

Bring in daily close for AAPL for every transaction, iterate through list and bring in daily close price for transaction date and 30 days into the future.

In [8]:
msft_filtered_df=tickers_df[(tickers_df['ticker']=='MSFT') & (tickers_df['amount'])]
# display(msft_filtered_df)

aapl_filtered_df=tickers_df[(tickers_df['ticker']=='AAPL') & (tickers_df['amount'])]
# display(aapl_filtered_df)

bac_filtered_df=tickers_df[(tickers_df['ticker']=='BAC') & (tickers_df['amount'])]
# display(bac_filtered_df)

dis_filtered_df=tickers_df[(tickers_df['ticker']=='DIS') & (tickers_df['amount'])]
# display(dis_filtered_df)

pfe_filtered_df=tickers_df[(tickers_df['ticker']=='PFE') & (tickers_df['amount'])]
# display(pfe_filtered_df)

combined_data=pd.concat([msft_filtered_df, aapl_filtered_df, bac_filtered_df, dis_filtered_df, pfe_filtered_df], axis=0)
display(combined_data)


,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
301,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
309,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
...,...,...,...,...,...
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7326,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [9]:
dup_drop=combined_data.drop_duplicates()
dup_drop

,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,9/16/2015,PFE,Purchase,"$15,000",Gary C. Peters
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [10]:
ready_csv_data=dup_drop.dropna()#.set_index('transaction_date')
display(ready_csv_data)



,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,9/16/2015,PFE,Purchase,"$15,000",Gary C. Peters
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


This is the API for stock data that I will concat to the existing df, I will iterate through each row and input the ticker symbol into the API 

Use top 5 tickers invested

In [11]:
# duration=180
# for idx, row in senator_trades_df.iterrows(): 
#     start=row['transaction_date'] # 2020-01-03
#     ticker=row['ticker'] # AAPL
#     df_in_timeframe=yf_data.loc[start:start+duration, ticker]
#     return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
#     senator_trades_df.loc[idx, f'{duration}_days_return']=return_in_timeframe

Concat the 5 tickers close prices into one df and input into the for loop for yf_data

In [12]:
import yfinance as yf
from datetime import datetime, timedelta

# 6 years of data
nine_year_end_date = datetime.today()
nine_year_start_date = nine_year_end_date - timedelta(days=365*9)

# # 30 days of data
# month_end_date = datetime.today()
# month_start_date = end_date - timedelta(days=30)

# # 90 days of data
# three_month_end_date = datetime.today()
# three_month_start_date = end_date - timedelta(days=90)

# # 180 days of data
# six_month_end_date = datetime.today()
# six_month_start_date = six_month_end_date - timedelta(days=180)

msft_tickers = yf.Tickers('MSFT')
msft_data=msft_tickers.tickers['MSFT'].history(start=nine_year_start_date, end=nine_year_end_date)
msft_ticker_df=pd.DataFrame(msft_data)
msft_ticker_df.index=msft_ticker_df.index.date
# msft_ticker_df

aapl_tickers=yf.Tickers('AAPL')
aapl_data=aapl_tickers.tickers['AAPL'].history(start=nine_year_start_date, end=nine_year_end_date)
aapl_ticker_df=pd.DataFrame(aapl_data)
aapl_ticker_df.index=aapl_ticker_df.index.date
# aapl_ticker_df


bac_tickers=yf.Tickers('BAC')
bac_data=bac_tickers.tickers['BAC'].history(start=nine_year_start_date, end=nine_year_end_date)
bac_ticker_df=pd.DataFrame(bac_data)
bac_ticker_df.index=bac_ticker_df.index.date
# bac_ticker_df

dis_tickers=yf.Tickers('DIS')
dis_data=dis_tickers.tickers['DIS'].history(start=nine_year_start_date, end=nine_year_end_date)
dis_ticker_df=pd.DataFrame(dis_data)
dis_ticker_df.index=dis_ticker_df.index.date
# dis_ticker_df

pfe_tickers=yf.Tickers('PFE')
pfe_data=pfe_tickers.tickers['PFE'].history(start=nine_year_start_date, end=nine_year_end_date)
pfe_ticker_df=pd.DataFrame(pfe_data)
pfe_ticker_df.index=pfe_ticker_df.index.date
# pfe_ticker_df

spy_tickers=yf.Tickers('SPY')
spy_data=spy_tickers.tickers['SPY'].history(start=nine_year_start_date, end=nine_year_end_date)
spy_ticker_df=pd.DataFrame(spy_data)
spy_ticker_df.index=spy_ticker_df.index.date
# spy_ticker_df

In [25]:
msft_filtered_ticker_df=msft_ticker_df[['Close']]
# msft_filtered_ticker_df

aapl_filtered_ticker_df=aapl_ticker_df[['Close']]
# display(aapl_filtered_ticker_df)

bac_filtered_ticker_df=bac_ticker_df[['Close']]
# display(bac_filtered_ticker_df)


dis_filtered_ticker_df=dis_ticker_df[['Close']]
# display(dis_filtered_ticker_df)


pfe_filtered_ticker_df=pfe_ticker_df[['Close']]
# display(pfe_filtered_ticker_df)

spy_filtered_ticker_df=spy_ticker_df[['Close']]
# display(spy_filtered_ticker_df)


ready_ticker_data=pd.concat([msft_filtered_ticker_df, aapl_filtered_ticker_df, bac_filtered_ticker_df, dis_filtered_ticker_df, pfe_filtered_ticker_df, spy_filtered_ticker_df], axis=1, keys=['MSFT', 'AAPL', 'BAC', 'DIS', 'PFE', 'S&P500']).dropna()
display(ready_ticker_data)

,MSFT,AAPL,BAC,DIS,PFE,S&P500
,Close,Close,Close,Close,Close,Close
2014-08-11,37.061378,21.445505,12.801788,80.527657,19.267244,164.482162
2014-08-12,37.335907,21.441040,12.793376,80.260780,19.151293,164.253067
2014-08-13,37.816334,21.724773,12.827019,80.619690,19.239962,165.364868
2014-08-14,37.979328,21.782860,12.885900,81.549210,19.594616,166.145706
2014-08-15,38.425446,21.890106,12.801788,82.165825,19.533232,166.111740
...,...,...,...,...,...,...
2023-07-31,335.920013,196.449997,32.000000,88.889999,36.060001,457.790009
2023-08-01,336.339996,195.610001,31.620001,89.029999,35.610001,456.480011
2023-08-02,327.500000,192.580002,31.190001,86.300003,35.349998,450.130005


In [26]:
ready_ticker_data.columns

MultiIndex([(  'MSFT', 'Close'),
            (  'AAPL', 'Close'),
            (   'BAC', 'Close'),
            (   'DIS', 'Close'),
            (   'PFE', 'Close'),
            ('S&P500', 'Close')],
           )

In [27]:
new_cols=[col for col, close_col in ready_ticker_data.columns]

ready_ticker_data.columns=new_cols

ready_ticker_data.head()

,MSFT,AAPL,BAC,DIS,PFE,S&P500
2014-08-11,37.061378,21.445505,12.801788,80.527657,19.267244,164.482162
2014-08-12,37.335907,21.441040,12.793376,80.260780,19.151293,164.253067
2014-08-13,37.816334,21.724773,12.827019,80.619690,19.239962,165.364868
2014-08-14,37.979328,21.782860,12.885900,81.549210,19.594616,166.145706
2014-08-15,38.425446,21.890106,12.801788,82.165825,19.533232,166.111740


In [28]:
import pandas as pd
from datetime import timedelta

ready_csv_data['transaction_date'] = pd.to_datetime(ready_csv_data['transaction_date'])

duration_list = [30, 60, 90, 180]
for duration in duration_list:
    for idx, row in ready_csv_data.iterrows():
        start = row['transaction_date']
        ticker = row['ticker']

        if ticker not in ready_ticker_data.columns:
            continue

        end = start + timedelta(days=duration)
        df_in_timeframe = ready_ticker_data.loc[start:end, [ticker, 'S&P500']]

        if df_in_timeframe.empty:
            continue

        return_in_timeframe = df_in_timeframe.iloc[-1, 0] / df_in_timeframe.iloc[0, 0]
        sp_return_in_timeframe = df_in_timeframe.iloc[-1, 1] / df_in_timeframe.iloc[0, 1]
        excess_return = return_in_timeframe - sp_return_in_timeframe
        ready_csv_data.loc[idx, f'{duration}_days_excess_return'] = excess_return.item()

display(ready_csv_data)


TypeError: Cannot compare Timestamp with datetime.date. Use ts == pd.Timestamp(date) or ts.date() == date instead.

In [17]:
# ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

# duration=60
# for idx, row in ready_csv_data.iterrows(): 
#     start=row['transaction_date'] # 2020-01-03
#     ticker=row['ticker'] # AAPL
    
#     if ticker not in ready_ticker_data.columns:
#         continue
        
#     end =start+timedelta(days=duration)
#     df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
#     if df_in_timeframe.empty:
#         continue
        
#     return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
#     ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

# display(ready_csv_data)

In [18]:
# ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

# duration=90
# for idx, row in ready_csv_data.iterrows(): 
#     start=row['transaction_date'] # 2020-01-03
#     ticker=row['ticker'] # AAPL
    
#     if ticker not in ready_ticker_data.columns:
#         continue
        
#     end =start+timedelta(days=duration)
#     df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
#     if df_in_timeframe.empty:
#         continue

        
#     return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
#     ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

# display(ready_csv_data)

In [19]:
# ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

# duration=180
# for idx, row in ready_csv_data.iterrows(): 
#     start=row['transaction_date'] # 2020-01-03
#     ticker=row['ticker'] # AAPL
    
#     if ticker not in ready_ticker_data.columns:
#         continue
        
#     end =start+timedelta(days=duration)
#     df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
#     if df_in_timeframe.empty:
#         continue
        
#     return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
#     ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

# display(ready_csv_data)

In [20]:
display(ready_csv_data.head(50))

,transaction_date,ticker,type,amount,senator
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville
326,2022-11-22,MSFT,Sale (Partial),"$250,000",Tommy Tuberville
359,2022-10-28,MSFT,Sale (Full),"$15,000",Tommy Tuberville
363,2022-10-24,MSFT,Purchase,"$50,000",Tommy Tuberville
365,2022-10-13,MSFT,Purchase,"$500,000",Tommy Tuberville
402,2022-10-05,MSFT,Sale (Partial),"$250,000",Tommy Tuberville


In [32]:
ready_csv_data[ready_csv_data['senator']=='Angus S. King, Jr.']

,transaction_date,ticker,type,amount,senator
5519,2017-05-03,DIS,Sale (Partial),"$15,000","Angus S. King, Jr."


In [33]:
# ready_csv_data_indicator[ready_csv_data_indicator['senator']=='Angus S. King, Jr.']

In [35]:
ready_csv_data_indicator=ready_csv_data.copy()
indicator_mapping={'Sale (Partial)': -1, 'Purchase': 1, 'Sale (Full)': -1, 'Exchange': 0}
ready_csv_data_indicator['indicator']=ready_csv_data_indicator['type'].replace(indicator_mapping)
ready_csv_data_indicator[['30_days_excess_return', '60_days_excess_return', '90_days_excess_return', '180_days_excess_return']]=ready_csv_data_indicator[['30_days_excess_return', '60_days_excess_return', '90_days_excess_return', '180_days_excess_return']].multiply(ready_csv_data['indicator'], axis=0)>0


KeyError: "None of [Index(['30_days_excess_return', '60_days_excess_return',\n       '90_days_excess_return', '180_days_excess_return'],\n      dtype='object')] are in the [columns]"

In [36]:
ready_csv_data_indicator.groupby('amount').mean()

TypeError: Could not convert MSFTMSFTMSFTMSFTMSFTAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLAAPLBACDISDISDISDISDISDISDISPFEPFE to numeric

In [ ]:
ready_csv_data.head()

In [ ]:
ready_csv_data.groupby(['senator', 'type']).mean().sort_values('180_days_excess_return', ascending=False)

In [ ]:
ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])
ready_csv_data.dtypes

In [31]:
ready_csv_data.hvplot.bar(x='transaction_date', y='180_days_excess_return', color='type', width=1000, height=500, cmap=['Red', 'Green', 'Violet', 'Grey'])

:Bars   [transaction_date]   (180_days_excess_return,type)

In [64]:
# data_to_plot.hvplot.bar(x='senator', y='30_days_excess_return', color='type', width=1000, height=500, cmap=['Blue', 'Yellow', 'Orange', 'Red'])

NameError: name 'data_to_plot' is not defined

In [ ]:
ready_ticker_data.to_csv('/Users/matt/Desktop/senator_trading_performance/ready_ticker_data.csv', index=False)

pie chart compare net positve and negative trades. Groupby type of day.